# Clustering with 2023 data ONLY

Try with only 2023 data

In [1]:
!pip install scipy==1.11.4

In [2]:
from calitp_data_analysis.sql import query_sql
from utils_transit_peer_groups import *
import sys
import altair as alt
import pandas as pd

sys.path.append("../ntd/monthly_ridership_report")
from update_vars import GCS_FILE_PATH, NTD_MODES, NTD_TOS

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

In [4]:
fixed_guideway = [
    "SR",
    "HR",
    "YR",
    "CR",
    "CC",
    "LR",
    "MG",
    "TB",
]

nonfixed_guideway = [
    "DR",
    "MB",
    "DT",
    "CB",
    "VP",
    "RB",
]

other_modes = [
    "FB",
    "OT", # no clue what this is, not in NTD glossary
    "OR", # same with this
    
]

mode_dict ={
    "SR":"fixed_guideway",
    "HR":"fixed_guideway",
    "YR":"fixed_guideway",
    "CR":"fixed_guideway",
    "CC":"fixed_guideway",
    "LR":"fixed_guideway",
    "MG":"fixed_guideway",
    "TB":"fixed_guideway",
    "DR":"nonfixed_guideway",
    "MB":"nonfixed_guideway",
    "DT":"nonfixed_guideway",
    "CB":"nonfixed_guideway",
    "VP":"nonfixed_guideway",
    "RB":"nonfixed_guideway",
    "FB":"other_mode",
    "OT":"other_mode", # no clue what this is, not in NTD glossary
    "OR":"other_mode",
}

In [5]:
#ID cols
id_cols = [
    "ntd_id",
    "source_agency",
    "city",
    "primary_uza_name"
]
# include in clustering
categorical_cols = ["mode_cat", "service", "reporter_type"]

# include in clustering
numerical_cols = [
    "total_upt",
    "total_vrh",
    "total_vrm",
    "opexp_total",
    "total_voms",
    # "uza_population",
    # "uza_area_sq_miles",
]

In [6]:
# alt method to query warehous data, via cal-itp docs
query = f"""
        SELECT
          upt.ntd_id,
          upt.source_agency,
          upt.agency_status,
          upt.city,
          upt.primary_uza_name,
          upt.uza_population,
          upt.uza_area_sq_miles,
          upt.year,
          upt.mode,
          upt.service,
          upt.reporter_type,
          SUM(upt.upt) AS total_upt,
          SUM(voms.voms) AS total_voms,
          SUM(vrh.vrh) AS total_vrh,
          SUM(vrm.vrm) AS total_vrm,
          SUM(opexp_total.opexp_total) AS opexp_total
        FROM
          cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_upt AS upt
        INNER JOIN
          cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_voms AS voms
        ON
          upt.ntd_id = voms.ntd_id
          AND upt.year = voms.year
          AND upt.source_agency = voms.source_agency
          AND upt.agency_status = voms.agency_status
          AND upt.primary_uza_name = voms.primary_uza_name
          AND upt.uza_population = voms.uza_population
          AND upt.uza_area_sq_miles = voms.uza_area_sq_miles
        INNER JOIN
          cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_vrh AS vrh
        ON
          upt.ntd_id = vrh.ntd_id
          AND upt.year = vrh.year
          AND upt.source_agency = vrh.source_agency
          AND upt.agency_status = vrh.agency_status
          AND upt.primary_uza_name = vrh.primary_uza_name
          AND upt.uza_population = vrh.uza_population
          AND upt.uza_area_sq_miles = vrh.uza_area_sq_miles
        INNER JOIN
          cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_vrm AS vrm
        ON
          upt.ntd_id = vrm.ntd_id
          AND upt.year = vrm.year
          AND upt.source_agency = vrm.source_agency
          AND upt.agency_status = vrm.agency_status
          AND upt.primary_uza_name = vrm.primary_uza_name
          AND upt.uza_population = vrm.uza_population
          AND upt.uza_area_sq_miles = vrm.uza_area_sq_miles
        INNER JOIN
          cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_opexp_total AS opexp_total
        ON
          upt.ntd_id = opexp_total.ntd_id
          AND upt.year = opexp_total.year
          AND upt.source_agency = opexp_total.source_agency
          AND upt.agency_status = opexp_total.agency_status
          AND upt.primary_uza_name = opexp_total.primary_uza_name
          AND upt.uza_population = opexp_total.uza_population
          AND upt.uza_area_sq_miles = opexp_total.uza_area_sq_miles
        WHERE
          upt.source_state = "CA"
          AND upt.year = 2023
        GROUP BY
          upt.ntd_id,
          upt.source_agency,
          upt.agency_status,
          upt.city,
          upt.year,
          upt.primary_uza_name,
          upt.uza_population,
          upt.uza_area_sq_miles,
          upt.mode,
          upt.service,
          upt.reporter_type
        """

ntd_2023_data = query_sql(query).fillna(0)

In [7]:
ntd_2023_data[numerical_cols] = ntd_2023_data[numerical_cols].astype("int64", errors="ignore")

## replace values for mode and tos

In [8]:
service_dict = {
    "TX":"PT",
    "TN":"PT"
}

ntd_2023_data["service"] = ntd_2023_data["service"].replace(service_dict)

In [9]:
ntd_2023_data["service"].unique()

array(['PT', 'DO'], dtype=object)

In [10]:
ntd_2023_data["mode_cat"] = ntd_2023_data["mode"].map(mode_dict)

In [11]:
ntd_2023_data["mode_cat"].unique()

array(['nonfixed_guideway', 'fixed_guideway', 'other_mode', nan],
      dtype=object)

In [20]:
ntd_2023_data.head()

,ntd_id,source_agency,agency_status,city,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,service,reporter_type,total_upt,total_voms,total_vrh,total_vrm,opexp_total,mode_cat
0,90198,City of Porterville (COLT) - Transit Department,Active,Porterville,"Porterville, CA",69862,16.35,2023,MB,PT,Building Reporter,0,0,0,0,0,nonfixed_guideway
1,90198,City of Porterville (COLT) - Transit Department,Active,Porterville,"Porterville, CA",69862,16.35,2023,DR,PT,Building Reporter,0,0,0,0,0,nonfixed_guideway
2,90036,Orange County Transportation Authority (OCTA),Active,Orange,"Los Angeles--Long Beach--Anaheim, CA",12237376,1636.83,2023,SR,PT,Full Reporter,0,1075448,2764357893,37310649233,428923289696,fixed_guideway
3,90013,Santa Clara Valley Transportation Authority (VTA),Active,San Jose,"San Jose, CA",1837446,285.48,2023,HR,DO,Full Reporter,0,262144,887382528,11150999040,232484337152,fixed_guideway
4,90170,ATC / Vancom,Inactive,0,"San Francisco--Oakland, CA",3515933,513.80,2023,DR,PT,Full Reporter,0,0,0,0,0,nonfixed_guideway


## 2023 - explore data

In [14]:
display(
    ntd_2023_data.info(),

    ntd_2023_data["service"].value_counts(),

    ntd_2023_data["mode_cat"].value_counts(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ntd_id             667 non-null    object 
 1   source_agency      667 non-null    object 
 2   agency_status      667 non-null    object 
 3   city               667 non-null    object 
 4   primary_uza_name   667 non-null    object 
 5   uza_population     667 non-null    int64  
 6   uza_area_sq_miles  667 non-null    float64
 7   year               667 non-null    int64  
 8   mode               667 non-null    object 
 9   service            667 non-null    object 
 10  reporter_type      667 non-null    object 
 11  total_upt          667 non-null    int64  
 12  total_voms         667 non-null    int64  
 13  total_vrh          667 non-null    int64  
 14  total_vrm          667 non-null    int64  
 15  opexp_total        667 non-null    int64  
 16  mode_cat           666 non

None

PT    437
DO    230
Name: service, dtype: int64

nonfixed_guideway    622
fixed_guideway        28
other_mode            16
Name: mode_cat, dtype: int64

## 2023 - correlation matrix

In [ ]:
ntd_2023_data[numerical_cols].corr()

## 2023 - Manual Hierarchal clustering w/ ward

In [15]:
from sklearn.cluster import AgglomerativeClustering 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [16]:
numerical_cols

['total_upt', 'total_vrh', 'total_vrm', 'opexp_total', 'total_voms']

In [17]:
categorical_cols

['mode_cat', 'service', 'reporter_type']

In [18]:
# 1. set up pre-processing steps with column transformer

preprocessor = ColumnTransformer(
    [
        ("ntd_metrics", StandardScaler(), numerical_cols),
        ("categorical", OneHotEncoder(drop="first", sparse_output=False), categorical_cols)
    ]
)
preprocessor

,transformers,"[('ntd_metrics', ...), ('categorical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [19]:
# 2. set up pipeline. First pre-processing, then clustering
pipeline = Pipeline(
    [
        ("preprocessing", preprocessor),
        ("clustering", AgglomerativeClustering(n_clusters=10, linkage="ward"))
    ]
)

pipeline

,steps,"[('preprocessing', ...), ('clustering', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('ntd_metrics', ...), ('categorical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Clustering with entire raw data 

In [21]:
# 3. use pipeline to fit clustering model. create new column for clustering
ntd_2023_fit = ntd_2023_data.copy() # why do i need to copy/clone?

ntd_2023_fit["cluster_name"] = pipeline.fit_predict(ntd_2023_fit)

In [22]:
display(
    ntd_2023_fit["cluster_name"].value_counts(),
    ntd_2023_fit[["source_agency","total_upt","total_vrh","cluster_name"]].sample(3)
)

8    183
1    182
2    128
0     83
9     50
4     22
3     10
6      7
5      1
7      1
Name: cluster_name, dtype: int64

,source_agency,total_upt,total_vrh,cluster_name
508,Los Angeles County - Department of Public Work...,20089,6116,1
655,Yosemite Area Regional Transportation System (...,991200,138920,2
538,Morongo Basin Transit Authority (MBTA BUS),11810934,860409,2


In [23]:
ntd_2023_data["service"].unique()

array(['PT', 'DO'], dtype=object)

### what are the feature columns with onehotencoding drop=first?

In [24]:
feature_names = preprocessor.get_feature_names_out()
display(list(feature_names))

['ntd_metrics__total_upt',
 'ntd_metrics__total_vrh',
 'ntd_metrics__total_vrm',
 'ntd_metrics__opexp_total',
 'ntd_metrics__total_voms',
 'categorical__mode_cat_nonfixed_guideway',
 'categorical__mode_cat_other_mode',
 'categorical__mode_cat_nan',
 'categorical__service_PT',
 'categorical__reporter_type_Full Reporter',
 'categorical__reporter_type_Reduced Reporter',
 'categorical__reporter_type_Rural Reporter',
 'categorical__reporter_type_Separate Service']

### what are the feature columns with `onehotencoding drop=None`?


In [25]:
preprocessor_2 = ColumnTransformer(
    [
        ("ntd_metrics", StandardScaler(), numerical_cols),
        ("categorical", OneHotEncoder(
            # drop="first", 
            sparse_output=False
        ), categorical_cols)
    ]
)

pipeline_2 = Pipeline(
    [
        ("preprocessing", preprocessor_2),
        ("clustering", AgglomerativeClustering(n_clusters=10, linkage="ward"))
    ]
)

ntd_2023_fit_2 = ntd_2023_data.copy()

ntd_2023_fit_2["cluster"] = pipeline_2.fit_predict(ntd_2023_fit_2)

In [26]:
feature_names_2= preprocessor_2.get_feature_names_out()

display(list(feature_names_2)) # service_DO appears

['ntd_metrics__total_upt',
 'ntd_metrics__total_vrh',
 'ntd_metrics__total_vrm',
 'ntd_metrics__opexp_total',
 'ntd_metrics__total_voms',
 'categorical__mode_cat_fixed_guideway',
 'categorical__mode_cat_nonfixed_guideway',
 'categorical__mode_cat_other_mode',
 'categorical__mode_cat_nan',
 'categorical__service_DO',
 'categorical__service_PT',
 'categorical__reporter_type_Building Reporter',
 'categorical__reporter_type_Full Reporter',
 'categorical__reporter_type_Reduced Reporter',
 'categorical__reporter_type_Rural Reporter',
 'categorical__reporter_type_Separate Service']

## Using utils function to cluster

### 2023 data, all modes, PT/DO service

In [27]:
cluster_2023 = make_hierarchal_clustering(
    data = ntd_2023_data,
    num_cols = numerical_cols,
    cat_cols = categorical_cols,
    cluster_num = 10
    
)

for i in cluster_2023["cluster_name"].unique():
    print(f"""
    \n How many unique agencies are in Cluster {i}: {cluster_2023[cluster_2023['cluster_name']==i]["source_agency"].nunique()}
    Cluster {i} sample:""")
    display(cluster_2023[cluster_2023["cluster_name"]==i].head()),
    print(f"\nCluster {i} summary stats:")
    display(cluster_2023[cluster_2023["cluster_name"]==i].describe())



    
 How many unique agencies are in Cluster 2: 52
    Cluster 2 sample:


,ntd_id,source_agency,agency_status,city,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,service,reporter_type,total_upt,total_voms,total_vrh,total_vrm,opexp_total,mode_cat,cluster_name
0,90198,City of Porterville (COLT) - Transit Department,Active,Porterville,"Porterville, CA",69862,16.35,2023,MB,PT,Building Reporter,0,0,0,0,0,nonfixed_guideway,2
1,90198,City of Porterville (COLT) - Transit Department,Active,Porterville,"Porterville, CA",69862,16.35,2023,DR,PT,Building Reporter,0,0,0,0,0,nonfixed_guideway,2
9,91063,Calaveras County Department of Public Works,Inactive,0,California Non-UZA,4357850,0.00,2023,MB,PT,Rural Reporter,0,0,0,0,0,nonfixed_guideway,2
15,91018,City of Arcata (A&MRTS),Active,Arcata,California Non-UZA,4357850,0.00,2023,MB,DO,Rural Reporter,0,16,45384,609936,8749840,nonfixed_guideway,2
25,91071,City of Chowchilla (CATX) - Transit Department,Active,Chowchilla,California Non-UZA,4357850,0.00,2023,DR,PT,Rural Reporter,0,54,55350,527958,14011704,nonfixed_guideway,2



Cluster 2 summary stats:


,uza_population,uza_area_sq_miles,year,total_upt,total_voms,total_vrh,total_vrm,opexp_total,cluster_name
count,1.280000e+02,128.000000,128.0,1.280000e+02,128.000000,1.280000e+02,1.280000e+02,1.280000e+02,128.0
mean,4.182396e+06,15.391250,2023.0,5.340320e+06,532.250000,7.948250e+05,1.410864e+07,8.499376e+07,2.0
std,7.635585e+05,81.773353,0.0,2.311191e+07,999.613256,1.291742e+06,2.479946e+07,1.275380e+08,0.0
min,6.986200e+04,0.000000,2023.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.0
25%,4.357850e+06,0.000000,2023.0,0.000000e+00,14.500000,2.686000e+04,4.367460e+05,3.131456e+06,2.0
50%,4.357850e+06,0.000000,2023.0,6.387200e+04,136.000000,1.900015e+05,5.315841e+06,3.060345e+07,2.0
75%,4.357850e+06,0.000000,2023.0,1.106420e+06,486.000000,7.642080e+05,1.615850e+07,1.210012e+08,2.0
max,4.357850e+06,467.580000,2023.0,2.164316e+08,4160.000000,4.746250e+06,1.059334e+08,4.552014e+08,2.0



    
 How many unique agencies are in Cluster 3: 1
    Cluster 3 sample:


...

,uza_population,uza_area_sq_miles,year,total_upt,total_voms,total_vrh,total_vrm,opexp_total,cluster_name
count,1.0,1.00,1.0,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00,1.0
mean,12237376.0,1636.83,2023.0,8.055469e+11,1357312.0,4.208722e+09,5.604735e+10,1.124672e+12,5.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,12237376.0,1636.83,2023.0,8.055469e+11,1357312.0,4.208722e+09,5.604735e+10,1.124672e+12,5.0
25%,12237376.0,1636.83,2023.0,8.055469e+11,1357312.0,4.208722e+09,5.604735e+10,1.124672e+12,5.0
50%,12237376.0,1636.83,2023.0,8.055469e+11,1357312.0,4.208722e+09,5.604735e+10,1.124672e+12,5.0
75%,12237376.0,1636.83,2023.0,8.055469e+11,1357312.0,4.208722e+09,5.604735e+10,1.124672e+12,5.0
max,12237376.0,1636.83,2023.0,8.055469e+11,1357312.0,4.208722e+09,5.604735e+10,1.124672e+12,5.0



    
 How many unique agencies are in Cluster 7: 1
    Cluster 7 sample:


,ntd_id,source_agency,agency_status,city,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,service,reporter_type,total_upt,total_voms,total_vrh,total_vrm,opexp_total,mode_cat,cluster_name
554,90036,Orange County Transportation Authority (OCTA),Active,Orange,"Los Angeles--Long Beach--Anaheim, CA",12237376,1636.83,2023,MB,DO,Full Reporter,331437877573,1075448,2764357893,37310649233,428923289696,nonfixed_guideway,7



Cluster 7 summary stats:


,uza_population,uza_area_sq_miles,year,total_upt,total_voms,total_vrh,total_vrm,opexp_total,cluster_name
count,1.0,1.00,1.0,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00,1.0
mean,12237376.0,1636.83,2023.0,3.314379e+11,1075448.0,2.764358e+09,3.731065e+10,4.289233e+11,7.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,12237376.0,1636.83,2023.0,3.314379e+11,1075448.0,2.764358e+09,3.731065e+10,4.289233e+11,7.0
25%,12237376.0,1636.83,2023.0,3.314379e+11,1075448.0,2.764358e+09,3.731065e+10,4.289233e+11,7.0
50%,12237376.0,1636.83,2023.0,3.314379e+11,1075448.0,2.764358e+09,3.731065e+10,4.289233e+11,7.0
75%,12237376.0,1636.83,2023.0,3.314379e+11,1075448.0,2.764358e+09,3.731065e+10,4.289233e+11,7.0
max,12237376.0,1636.83,2023.0,3.314379e+11,1075448.0,2.764358e+09,3.731065e+10,4.289233e+11,7.0


### 2023 data, non-fixed route modes , PT/DO service

In [37]:
non_fixed_route_data = ntd_2023_data[
    ntd_2023_data["mode_cat"]=="nonfixed_guideway"
].groupby(id_cols).agg({
    i:"sum" for i in numerical_cols
}).reset_index()

display(
    non_fixed_route_data["ntd_id"].value_counts()[non_fixed_route_data["ntd_id"].value_counts()>1], # empty, there are no duplicate ntd_ids
    non_fixed_route_data.head()
)

Series([], Name: ntd_id, dtype: int64)

,ntd_id,source_agency,city,primary_uza_name,total_upt,total_vrh,total_vrm,opexp_total,total_voms
0,90003,San Francisco Bay Area Rapid Transit District ...,Oakland,"San Francisco--Oakland, CA",0,681018500,21308437250,195646239500,145500
1,90004,Golden Empire Transit District (GET),Bakersfield,"Bakersfield, CA",266781033,23436378,317845296,2960905266,7209
2,90006,Santa Cruz Metropolitan Transit District (SCMTD),Santa Cruz,"Santa Cruz, CA",4341633696,278313408,3855763296,62945840016,120528
3,90007,City of Modesto (MAX),0,"Modesto, CA",0,0,0,0,0
4,90008,City of Santa Monica (BBB) - Department of Tra...,Santa Monica,"Los Angeles--Long Beach--Anaheim, CA",4854828125,260590000,2450246875,50485795000,89375


In [ ]:
non_fixed_route_2023 = make_hierarchal_clustering(
    data = non_fixed_route_data,
    num_cols = numerical_cols,
    cat_cols = categorical_cols,
    cluster_num = 10
    
)

z_nonfixed_route = make_dendrogram_data(
    data = non_fixed_route_data,
    num_cols = numerical_cols,
    cat_cols = categorical_cols
)


for i in non_fixed_route_2023["cluster_name"].unique():
    print(f"""
    \n How many unique agencies are in Cluster {i}: {non_fixed_route_2023[non_fixed_route_2023['cluster_name']==i]["source_agency"].nunique()}
    Cluster {i} sample:""")
    display(non_fixed_route_2023[non_fixed_route_2023["cluster_name"]==i].head()),
    print(f"\nCluster {i} summary stats:")
    display(non_fixed_route_2023[non_fixed_route_2023["cluster_name"]==i].describe())

    
# fig_nonfixed_route = ff.create_dendrogram(z_nonfixed_route, labels = non_fixed_route_data["source_agency"].tolist())
# fig_nonfixed_route.update_layout(width=800, height=500)
# fig_nonfixed_route.show()

plt.figure(figsize=(10, 5))
dendrogram(z_nonfixed_route, labels=non_fixed_route_data["ntd_id"].tolist(), leaf_rotation=90)
plt.title("Dendrogram (Ward's Method)")
plt.xlabel("ntd_id")
plt.ylabel("Distance")
plt.tight_layout()
plt.show()

### 2023 data, fixed route modes , PT/DO service

In [ ]:
fixed_route_data = ntd_2023_data[
    ntd_2023_data["mode"].isin(fixed_guideway)
].groupby(id_cols+categorical_cols).agg({
    i:"sum" for i in numerical_cols
}).reset_index()

fixed_route_data.head()

In [ ]:
fixed_route_2023 = make_hierarchal_clustering(
    data = fixed_route_data,
    num_cols = numerical_cols,
    cat_cols = categorical_cols,
    cluster_num = 10
    
)

z_fixed_route = make_dendrogram_data(
    data = fixed_route_data,
    num_cols = numerical_cols,
    cat_cols = categorical_cols
)

for i in fixed_route_2023["cluster_name"].unique():
    print(f"""
    \n How many unique agencies are in Cluster {i}: {fixed_route_2023[fixed_route_2023['cluster_name']==i]["source_agency"].nunique()}
    Cluster {i} sample:""")
    display(fixed_route_2023[fixed_route_2023["cluster_name"]==i].head())

# fig_fixed_route = ff.create_dendrogram(z_fixed_route,)
# fig_fixed_route.update_layout(width=800, height=500)
# fig_fixed_route.show()

plt.figure(figsize=(10, 5))
dendrogram(z_fixed_route, labels=fixed_route_data["ntd_id"].tolist(), leaf_rotation=90)
plt.title("Dendrogram (Ward's Method)")
plt.xlabel("ntd_id")
plt.ylabel("Distance")
plt.tight_layout()
plt.show()

### 2023 data, other modes, PT/DO service

In [ ]:
other_modes

In [ ]:


other_mode_2023 = make_hierarchal_clustering(
    data = ntd_2023_data[ntd_2023_data["mode"].isin(other_modes)],
    num_cols = numerical_cols,
    cat_cols = categorical_cols,
    cluster_num = 10
    
)

z_other_modes = make_dendrogram_data(
    data = ntd_2023_data[ntd_2023_data["mode"].isin(other_modes)],
    num_cols = numerical_cols,
    cat_cols = categorical_cols
)

for i in other_mode_2023["cluster_name"].unique():
    print(f"""
    \n How many unique agencies are in Cluster {i}: {other_mode_2023[other_mode_2023['cluster_name']==i]["source_agency"].nunique()}
    Cluster {i} sample:""")
    display(other_mode_2023[other_mode_2023["cluster_name"]==i].head()),
    print(f"\nCluster {i} summary stats:")
    display(other_mode_2023[other_mode_2023["cluster_name"]==i].describe())

    
fig_fixed_route = ff.create_dendrogram(z_other_modes)
fig_fixed_route.update_layout(width=800, height=500)
fig_fixed_route.show()

### non-fixed route, just DO


In [ ]:
non_fixed_route_do = ntd_2023_data[
    (ntd_2023_data["mode"].isin(nonfixed_guideway))
    & (ntd_2023_data["service"]=="DO")
].groupby(id_cols+categorical_cols).agg({
    i:"sum" for i in numerical_cols
}).reset_index()

display(
    non_fixed_route_do.info(),
    non_fixed_route_do.head()
)



In [ ]:
non_fixed_route_do_2023 = make_hierarchal_clustering(
    data = non_fixed_route_do,
    num_cols = numerical_cols,
    cat_cols = categorical_cols,
    cluster_num = 10
    
)

z_nonfixed_route_do = make_dendrogram_data(
    data = non_fixed_route_do,
    num_cols = numerical_cols,
    cat_cols = categorical_cols
)


for i in non_fixed_route_do_2023["cluster_name"].unique():
    print(f"""
    \n How many unique agencies are in Cluster {i}: {non_fixed_route_do_2023[non_fixed_route_do_2023['cluster_name']==i]["source_agency"].nunique()}
    Cluster {i} sample:""")
    display(non_fixed_route_do_2023[non_fixed_route_do_2023["cluster_name"]==i].head()),
    print(f"\nCluster {i} summary stats:")
    display(non_fixed_route_do_2023[non_fixed_route_do_2023["cluster_name"]==i].describe())

    
# fig_nonfixed_route = ff.create_dendrogram(z_nonfixed_route, labels = non_fixed_route_data["source_agency"].tolist())
# fig_nonfixed_route.update_layout(width=800, height=500)
# fig_nonfixed_route.show()

plt.figure(figsize=(10, 5))
dendrogram(z_nonfixed_route_do, labels=non_fixed_route_do["ntd_id"].tolist(), leaf_rotation=90)
plt.title("Dendrogram (Ward's Method)")
plt.xlabel("ntd_id")
plt.ylabel("Distance")
plt.tight_layout()
plt.show()